# Logical circuits creation

In [ ]:
L = 8
J = 1
h = 0.05
lamb = 0.5

particle_pair_left_position = 3
particle_pair_length = 1

final_time = 10
layers = 100
measure_every_layers = 1

import z2chain.circs as circs

logical_circs = circs.particle_pair_quench_simulation_circuits(L, J, h, lamb, particle_pair_left_position, particle_pair_length, final_time, layers, measure_every_layers)
# logical_circs[1].draw(output="mpl")

# Device selection

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService
from cobos.token import ibm_token

channel = "ibm_quantum"
device = "ibm_kyiv"

service = QiskitRuntimeService(channel=channel, token=ibm_token)
backend = service.get_backend(device)

# Circuit transpilation

In [ ]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

optimization_level = 1
qubit_inds = []